# INVENTORY PROBLEM

In [165]:
import numpy as np
import math

### PROBLEM SETUP

In [166]:
n_products = 3
n_raw_materials = 7

In [167]:
S = np.floor(100 * np.random.random(size=n_products)) + 1                   # Selling price for each product
A = np.floor(10 * np.random.random(size=(n_raw_materials, n_products)))     # Requirements matrix (7x3)
R = np.floor(100 * np.random.random(size=n_raw_materials)) + 1               # Upper bound on raw materials
C = np.floor(10 * np.random.random(size=n_raw_materials)) + 1               # Cost for each raw material
D = np.array([30, 30, 30])                                                  # Demand for each product

print(f"Selling Price: {S} \n Requirement Matrix:\n {A} \n SupplyBounds: {R} \n Cost for RawMaterials: {C} \n Demands : {D}")

Selling Price: [64. 99. 34.] 
 Requirement Matrix:
 [[9. 1. 0.]
 [0. 4. 0.]
 [5. 3. 3.]
 [6. 1. 7.]
 [5. 9. 3.]
 [9. 5. 9.]
 [0. 7. 8.]] 
 SupplyBounds: [ 6. 58. 81. 57.  7. 70.  7.] 
 Cost for RawMaterials: [ 9.  1.  2.  3.  7. 10.  2.] 
 Demands : [30 30 30]


In [168]:
net_profit = S - A.T @ C
print(f"Net profit per unit: {net_profit}\nThis is the negative of profit")

Net profit per unit: [-170.  -50. -120.]
This is the negative of profit


In [169]:
def bounded_coefficient_encoding(kappa_x, mu_x):
    """
    Algorithm 1: Bounded-Coefficient Encoding
    
    Args:
        kappa_x: upper bound on the integer variable x
        mu_x: upper bound on the coefficients of the encoding
    
    Returns:
        c_x: integer encoding coefficients
    """
    print(f"  Encoding variable with κ={kappa_x}, μ={mu_x}")
    
    # Check if we should use binary encoding (equation 5)
    if kappa_x < 2**(math.floor(math.log2(mu_x)) + 1):
        print(f"    Using binary encoding (κ < 2^⌊log(μ)⌋+1)")
        # Binary encoding with adjustment for κ
        log_kappa = math.floor(math.log2(kappa_x))
        binary_coeffs = [2**i for i in range(log_kappa + 1)]
        
        # Adjust last coefficient to exactly reach κ
        if sum(binary_coeffs[:-1]) < kappa_x:
            binary_coeffs[-1] = kappa_x - sum(binary_coeffs[:-1])
        
        print(f"    Coefficients: {binary_coeffs}")
        return binary_coeffs
    
    else:
        #print(f"    Using bounded-coefficient encoding")
        # Bounded-coefficient encoding (equation 6)
        rho = math.floor(math.log2(mu_x)) + 1
        nu = kappa_x - sum(2**(i-1) for i in range(1, rho + 1))
        eta = math.floor(nu / mu_x)
        
        #print(f"    ρ={rho}, ν={nu}, η={eta}")
        
        # Build coefficient vector
        c_x = []
        
        # First ρ coefficients: 2^(i-1) for i = 1, ..., ρ
        for i in range(1, rho + 1):
            c_x.append(2**(i-1))
        
        # Next η coefficients: μ_x
        for i in range(eta):
            c_x.append(mu_x)
        
        # Last coefficient if needed: ν - η*μ_x
        remainder = nu - eta * mu_x
        if remainder != 0:
            c_x.append(remainder)
        
        print(f"    Coefficients: {c_x}")
        return c_x

def create_bounded_encoding_system(kappa_vec, mu_vec):
    """Create the complete bounded coefficient encoding system for all variables"""
    n_variables = len(kappa_vec)
    
    
    # Get coefficients for each variable
    all_coefficients = []
    widths = []
    
    for i in range(n_variables):
        coeffs = bounded_coefficient_encoding(kappa_vec[i], mu_vec[i])
        all_coefficients.append(coeffs)
        widths.append(len(coeffs))
        print(f"Variable x[{i}]: κ={kappa_vec[i]}, μ={mu_vec[i]}, width={len(coeffs)}, coeffs={coeffs}\n")
    
    total_binary_vars = sum(widths)
    print(f"Total binary variables: {total_binary_vars}")
    
    return all_coefficients, widths, total_binary_vars

def create_bounded_encoding_matrix(n_variables, all_coefficients, widths):
    """Create encoding matrix C for bounded coefficient encoding where x = C @ y"""
    total_binary_vars = sum(widths)
    C = np.zeros((n_variables, total_binary_vars))
    
    start_idx = 0
    for i in range(n_variables):
        end_idx = start_idx + widths[i]
        C[i, start_idx:end_idx] = all_coefficients[i]
        start_idx = end_idx
    
    return C

In [170]:
kappa_vec, mu_vec = D.copy(), [8, 8, 8]

In [171]:
coefficients, widths, total_vars = create_bounded_encoding_system(kappa_vec, mu_vec)
all_coefficients = coefficients

  Encoding variable with κ=30, μ=8
    Coefficients: [1, 2, 4, 8, 8, np.int64(7)]
Variable x[0]: κ=30, μ=8, width=6, coeffs=[1, 2, 4, 8, 8, np.int64(7)]

  Encoding variable with κ=30, μ=8
    Coefficients: [1, 2, 4, 8, 8, np.int64(7)]
Variable x[1]: κ=30, μ=8, width=6, coeffs=[1, 2, 4, 8, 8, np.int64(7)]

  Encoding variable with κ=30, μ=8
    Coefficients: [1, 2, 4, 8, 8, np.int64(7)]
Variable x[2]: κ=30, μ=8, width=6, coeffs=[1, 2, 4, 8, 8, np.int64(7)]

Total binary variables: 18


In [172]:
C = create_bounded_encoding_matrix(n_products, coefficients, widths)

In [173]:
C.shape, C

((3, 18),
 array([[1., 2., 4., 8., 8., 7., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 2., 4., 8., 8., 7., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 2., 4., 8.,
         8., 7.]]))

In [174]:
def create_bounded_encoding_matrix(n_variables, all_coefficients, widths):
    """Create encoding matrix C for bounded coefficient encoding where x = C @ y"""
    total_binary_vars = sum(widths)
    C = np.zeros((n_variables, total_binary_vars))
    
    start_idx = 0
    for i in range(n_variables):
        end_idx = start_idx + widths[i]
        C[i, start_idx:end_idx] = all_coefficients[i]
        start_idx = end_idx
    
    return C

In [175]:
C = create_bounded_encoding_matrix(n_variables=3, all_coefficients=coefficients, widths = widths)
C

array([[1., 2., 4., 8., 8., 7., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 2., 4., 8., 8., 7., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 2., 4., 8.,
        8., 7.]])

### QUBO

In [176]:
def create_qubo_formulation(C, A, R, net_profit, penalty_multiplier=10):

    """
    Create QUBO formulation for the optimization problem
    
    Args:
        C: Encoding matrix (n_products x total_binary_vars)
        A: Requirements matrix (n_raw_materials x n_products) 
        R: Raw material limits (n_raw_materials,)
        net_profit: Net profit per unit (n_products,)
        penalty_multiplier: Multiplier for penalty weights
    
    Returns:
        Q: QUBO matrix (upper triangular)
        h: Linear coefficients
        penalty_weights: Penalty weights used
    """
    
    n_products, total_binary_vars = C.shape
    n_raw_materials = A.shape[0]
    
    # Step 1: Transform objective to binary space
    # Original: maximize Σⱼ net_profit[j] * x[j]
    # Binary: maximize (net_profit @ C) @ y
    # For minimization: minimize -(net_profit @ C) @ y
    
    linear_objective = -(net_profit @ C)  # Negative for minimization
    print(f"Linear objective coefficients: {linear_objective}")
    
    # Step 4: Choose penalty weights
    max_profit = np.max(np.abs(net_profit))
    penalty_weights = penalty_multiplier * max_profit * np.ones(n_raw_materials)
    print(f"Penalty weights: {penalty_weights}")
    print(f"Max profit magnitude: {max_profit}, multiplier: {penalty_multiplier}")
    
    # Step 2: Add quadratic penalty terms
    # Raw material constraints: A @ x ≤ R
    # In binary space: A @ C @ y ≤ R
    # Penalty: Σᵢ Pᵢ * ((A @ C @ y)[i] - R[i])²
    
    # Transform constraints to binary space
    A_binary = A @ C  # (n_raw_materials x total_binary_vars)
    print(f"Constraint matrix in binary space A_binary shape: {A_binary.shape}")
    
    # Step 3: Construct QUBO matrix
    # For penalty term Pᵢ * ((A_binary[i,:] @ y - R[i])²)
    # = Pᵢ * (y^T @ A_binary[i,:]^T @ A_binary[i,:] @ y - 2*R[i]*A_binary[i,:] @ y + R[i]²)
    
    Q = np.zeros((total_binary_vars, total_binary_vars))
    h = linear_objective.copy()
    
    for i in range(n_raw_materials):
        P_i = penalty_weights[i]
        a_i = A_binary[i, :]  # Constraint coefficients for raw material i
        
        # Quadratic terms: P_i * a_i^T @ a_i (outer product)
        Q += P_i * np.outer(a_i, a_i)
        
        # Linear terms: -2 * P_i * R[i] * a_i
        h -= 2 * P_i * R[i] * a_i
        
        # Constant term P_i * R[i]² is ignored (doesn't affect optimization)
    
    # Make Q upper triangular (QUBO convention)
    Q = np.triu(Q)
    
    print(f"QUBO matrix Q shape: {Q.shape}")
    print(f"Linear coefficients h shape: {h.shape}")
    print(f"Q matrix sparsity: {np.count_nonzero(Q)} / {Q.size} = {np.count_nonzero(Q)/Q.size:.3f}")
    
    return Q, h, penalty_weights

def verify_qubo_solution(y_binary, C, A, R, net_profit):
    """Verify a binary solution against original constraints"""
    # Decode binary solution
    x_solution = C @ y_binary
    
    print(f"\n=== SOLUTION VERIFICATION ===")
    print(f"Binary solution y: {y_binary}")
    print(f"Decoded solution x: {x_solution}")
    
    # Check demand constraints (should be automatically satisfied)
    demand_satisfied = np.all(x_solution <= D)
    print(f"Demand constraints satisfied: {demand_satisfied}")
    if not demand_satisfied:
        print(f"  Violations: {x_solution - D}")
    
    # Check raw material constraints
    raw_material_usage = A @ x_solution
    raw_material_satisfied = np.all(raw_material_usage <= R)
    print(f"Raw material constraints satisfied: {raw_material_satisfied}")
    print(f"Raw material usage: {raw_material_usage}")
    print(f"Raw material limits: {R}")
    if not raw_material_satisfied:
        violations = raw_material_usage - R
        print(f"  Violations: {violations}")
        print(f"  Violation indices: {np.where(violations > 0)[0]}")
    
    # Calculate objective value
    objective_value = net_profit @ x_solution
    print(f"Objective value (profit): {objective_value}")
    
    return x_solution, objective_value, demand_satisfied, raw_material_satisfied


In [177]:
Q, h, penalty_weights = create_qubo_formulation(C, A, R, net_profit, penalty_multiplier=10)

Linear objective coefficients: [ 170.  340.  680. 1360. 1360. 1190.   50.  100.  200.  400.  400.  350.
  120.  240.  480.  960.  960.  840.]
Penalty weights: [1700. 1700. 1700. 1700. 1700. 1700. 1700.]
Max profit magnitude: 170.0, multiplier: 10
Constraint matrix in binary space A_binary shape: (7, 18)
QUBO matrix Q shape: (18, 18)
Linear coefficients h shape: (18,)
Q matrix sparsity: 171 / 324 = 0.528


In [178]:
y_zero = np.zeros(total_vars, dtype=int)
x_zero, obj_zero, demand_ok, raw_ok = verify_qubo_solution(y_zero, C, A, R, net_profit)


=== SOLUTION VERIFICATION ===
Binary solution y: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Decoded solution x: [0. 0. 0.]
Demand constraints satisfied: True
Raw material constraints satisfied: True
Raw material usage: [0. 0. 0. 0. 0. 0. 0.]
Raw material limits: [ 6. 58. 81. 57.  7. 70.  7.]
Objective value (profit): 0.0


In [179]:
y_random = np.random.randint(0, 2, size=total_vars)
x_random, obj_random, demand_ok_r, raw_ok_r = verify_qubo_solution(y_random, C, A, R, net_profit)


=== SOLUTION VERIFICATION ===
Binary solution y: [0 1 1 1 1 0 0 1 1 1 0 1 0 0 0 0 1 0]
Decoded solution x: [22. 21.  8.]
Demand constraints satisfied: True
Raw material constraints satisfied: False
Raw material usage: [219.  84. 197. 209. 323. 375. 211.]
Raw material limits: [ 6. 58. 81. 57.  7. 70.  7.]
  Violations: [213.  26. 116. 152. 316. 305. 204.]
  Violation indices: [0 1 2 3 4 5 6]
Objective value (profit): -5750.0


### BRUTE FORCE SOLVING

In [180]:
def evaluate_qubo_objective(y, Q, h):
    """Evaluate QUBO objective function H(y) = y^T @ Q @ y + h^T @ y"""
    return y.T @ Q @ y + h.T @ y

def find_exact_minimum(Q, h, C, A, R, net_profit, max_evaluations=None):
    """
    Find the exact minimum of the QUBO problem through exhaustive search
    
    Args:
        Q: QUBO matrix
        h: Linear coefficients  
        C: Encoding matrix
        A: Requirements matrix
        R: Raw material limits
        net_profit: Net profit coefficients
        max_evaluations: Maximum number of solutions to evaluate (None = all)
    
    Returns:
        best_y: Optimal binary solution
        best_objective: Optimal QUBO objective value
        best_x: Optimal production quantities
        best_profit: Optimal profit value
        feasible_solutions: List of feasible solutions found
    """
    
    n_bits = len(h)
    total_combinations = 2**n_bits
    
    print(f"\n=== FINDING EXACT MINIMUM ===")
    print(f"Total binary variables: {n_bits}")
    print(f"Total combinations to evaluate: {total_combinations:,}")
    
    if max_evaluations is None:
        max_evaluations = total_combinations
    else:
        max_evaluations = min(max_evaluations, total_combinations)
    
    print(f"Evaluating: {max_evaluations:,} combinations")
    
    best_objective = float('inf')
    best_y = None
    best_x = None 
    best_profit = None
    feasible_solutions = []
    
    # Track progress
    progress_interval = max(1, max_evaluations // 20)
    
    for i in range(max_evaluations):
        # Convert integer to binary representation
        y = np.array([int(b) for b in format(i, f'0{n_bits}b')], dtype=int)
        
        # Evaluate QUBO objective
        objective = evaluate_qubo_objective(y, Q, h)
        
        # Decode to original variables
        x = C @ y
        
        # Check feasibility in original problem
        demand_feasible = np.all(x <= D)
        raw_material_usage = A @ x
        raw_material_feasible = np.all(raw_material_usage <= R)
        is_feasible = demand_feasible and raw_material_feasible
        
        # Calculate original profit
        profit = net_profit @ x
        
        # Store feasible solutions
        if is_feasible:
            feasible_solutions.append({
                'y': y.copy(),
                'x': x.copy(),
                'objective': objective,
                'profit': profit,
                'raw_usage': raw_material_usage.copy()
            })
        
        # Update best solution
        if objective < best_objective:
            best_objective = objective
            best_y = y.copy()
            best_x = x.copy()
            best_profit = profit
        
        # Progress update
        if (i + 1) % progress_interval == 0:
            pct = 100 * (i + 1) / max_evaluations
            print(f"  Progress: {pct:.1f}% ({i+1:,}/{max_evaluations:,})")
    
    print(f"\n=== EXACT SOLUTION RESULTS ===")
    print(f"Best QUBO objective: {best_objective}")
    print(f"Best binary solution: {best_y}")
    print(f"Best production quantities: {best_x}")
    print(f"Best profit: {best_profit}")
    print(f"Total feasible solutions found: {len(feasible_solutions)}")
    
    # Check if best solution is feasible
    x_best = C @ best_y
    demand_ok = np.all(x_best <= D)
    raw_usage = A @ x_best
    raw_ok = np.all(raw_usage <= R)
    
    print(f"Best solution feasibility:")
    print(f"  Demand constraints: {demand_ok}")
    print(f"  Raw material constraints: {raw_ok}")
    if not raw_ok:
        violations = raw_usage - R
        print(f"  Raw material violations: {violations}")
    
    return best_y, best_objective, best_x, best_profit, feasible_solutions

def analyze_feasible_solutions(feasible_solutions, top_k=10):
    """Analyze the top feasible solutions"""
    if not feasible_solutions:
        print("No feasible solutions found!")
        return
    
    print(f"\n=== ANALYZING TOP {min(top_k, len(feasible_solutions))} FEASIBLE SOLUTIONS ===")
    
    # Sort by profit (descending, since we want maximum profit)
    sorted_solutions = sorted(feasible_solutions, key=lambda x: x['profit'], reverse=True)
    
    for i, sol in enumerate(sorted_solutions[:top_k]):
        print(f"\nRank {i+1}:")
        print(f"  Production: {sol['x']}")
        print(f"  Profit: {sol['profit']:.2f}")
        print(f"  QUBO objective: {sol['objective']:.2f}")
        print(f"  Raw material usage: {sol['raw_usage']}")
        
        # Check utilization
        utilization = sol['raw_usage'] / R * 100
        print(f"  Raw material utilization %: {utilization}")

def quick_feasibility_check(max_evaluations=10000):
    """Quick check to see if any feasible solutions exist"""
    print(f"\n=== QUICK FEASIBILITY CHECK ===")
    print(f"Checking {max_evaluations:,} random solutions...")
    
    feasible_count = 0
    n_bits = len(h)
    
    for i in range(max_evaluations):
        # Random binary solution
        y = np.random.randint(0, 2, size=n_bits)
        x = C @ y
        
        # Check feasibility
        demand_ok = np.all(x <= D)
        raw_ok = np.all(A @ x <= R)
        
        if demand_ok and raw_ok:
            feasible_count += 1
            profit = net_profit @ x
            print(f"  Found feasible solution: x={x}, profit={profit:.2f}")
    
    print(f"Feasible solutions found: {feasible_count}/{max_evaluations} ({100*feasible_count/max_evaluations:.2f}%)")
    return feasible_count > 0

# Run quick feasibility check first
has_feasible = quick_feasibility_check(max_evaluations=10000)

if has_feasible:
    print("\nFeasible solutions exist! Proceeding with exact optimization...")
    # Find exact minimum (this may take a few minutes for 2^18 combinations)
    print("Warning: This may take 1-2 minutes for exhaustive search...")
    best_y, best_obj, best_x, best_profit, feasible_sols = find_exact_minimum(
        Q, h, C, A, R, net_profit, max_evaluations=None
    )
    
    # Analyze top feasible solutions
    analyze_feasible_solutions(feasible_sols, top_k=5)
    
else:
    print("\nNo feasible solutions found in random sampling!")
    print("The problem may be over-constrained. Consider:")
    print("1. Reducing demand requirements")
    print("2. Increasing raw material limits") 
    print("3. Adjusting penalty weights")
    
    # Still find the best solution even if infeasible
    print("\nFinding best solution (may be infeasible)...")
    best_y, best_obj, best_x, best_profit, _ = find_exact_minimum(
        Q, h, C, A, R, net_profit, max_evaluations=50000
    )


=== QUICK FEASIBILITY CHECK ===
Checking 10,000 random solutions...


  Found feasible solution: x=[0. 0. 0.], profit=0.00
Feasible solutions found: 1/10000 (0.01%)

Feasible solutions exist! Proceeding with exact optimization...

=== FINDING EXACT MINIMUM ===
Total binary variables: 18
Total combinations to evaluate: 262,144
Evaluating: 262,144 combinations
  Progress: 5.0% (13,107/262,144)
  Progress: 10.0% (26,214/262,144)
  Progress: 15.0% (39,321/262,144)
  Progress: 20.0% (52,428/262,144)
  Progress: 25.0% (65,535/262,144)
  Progress: 30.0% (78,642/262,144)
  Progress: 35.0% (91,749/262,144)
  Progress: 40.0% (104,856/262,144)
  Progress: 45.0% (117,963/262,144)
  Progress: 50.0% (131,070/262,144)
  Progress: 55.0% (144,177/262,144)
  Progress: 60.0% (157,284/262,144)
  Progress: 65.0% (170,391/262,144)
  Progress: 70.0% (183,498/262,144)
  Progress: 75.0% (196,605/262,144)
  Progress: 80.0% (209,712/262,144)
  Progress: 85.0% (222,819/262,144)
  Progress: 90.0% (235,926/262,144)
  Progress: 95.0% (249,033/262,144)
  Progress: 100.0% (262,140/262,1

### ANNEALING

In [181]:
import dimod
from neal import SimulatedAnnealingSampler

In [182]:
bqm = dimod.BQM.from_qubo(Q)

In [183]:
sampler = SimulatedAnnealingSampler()
objectives = []
samples = []


for i in range(10):
    sampleset = sampler.sample(bqm, num_reads=100)
    samples.append(sampleset.first)
    objectives.append(sampleset.first.energy)
    print(f"{i} run(s) done.")

np.mean(objectives), np.std(objectives)

0 run(s) done.
1 run(s) done.
2 run(s) done.
3 run(s) done.
4 run(s) done.
5 run(s) done.
6 run(s) done.
7 run(s) done.
8 run(s) done.
9 run(s) done.


(np.float64(0.0), np.float64(0.0))

In [184]:
import numpy as np
import math
import dimod
from neal import SimulatedAnnealingSampler

# Set random seed for reproducibility
np.random.seed(42)

# ===========================================
# PROBLEM SETUP WITH REALISTIC PARAMETERS
# ===========================================

print("=== CREATING REALISTIC PROBLEM INSTANCE ===")

n_products = 3
n_raw_materials = 7

# Generate more realistic problem data
S = np.array([100., 120., 80.])  # Higher selling prices
A = np.floor(5 * np.random.random(size=(n_raw_materials, n_products))) + 1  # Lower material requirements
R = np.array([100., 120., 80., 90., 150., 200., 180.])  # Higher material limits
C = np.floor(3 * np.random.random(size=n_raw_materials)) + 1  # Lower material costs
D = np.array([10, 10, 10])  # Lower demand limits

print(f"Selling prices S: {S}")
print(f"Raw material costs C: {C}")
print(f"Raw material limits R: {R}")
print(f"Demand limits D: {D}")
print(f"Requirements matrix A (raw_materials x products):\n{A}")

# Calculate net profit per unit
net_profit = S - A.T @ C
print(f"Net profit per unit: {net_profit}")

if np.all(net_profit < 0):
    print("WARNING: All profits are negative! Adjusting costs...")
    C = C * 0.3  # Reduce costs dramatically
    net_profit = S - A.T @ C
    print(f"Adjusted net profit per unit: {net_profit}")

# ===========================================
# ENCODING SYSTEM
# ===========================================

def bounded_coefficient_encoding(kappa_x, mu_x):
    """Bounded-Coefficient Encoding Algorithm"""
    print(f"  Encoding variable with κ={kappa_x}, μ={mu_x}")
    
    if kappa_x < 2**(math.floor(math.log2(mu_x)) + 1):
        print(f"    Using binary encoding")
        log_kappa = math.floor(math.log2(kappa_x))
        binary_coeffs = [2**i for i in range(log_kappa + 1)]
        
        if sum(binary_coeffs[:-1]) < kappa_x:
            binary_coeffs[-1] = kappa_x - sum(binary_coeffs[:-1])
        
        print(f"    Coefficients: {binary_coeffs}")
        return binary_coeffs
    else:
        print(f"    Using bounded-coefficient encoding")
        rho = math.floor(math.log2(mu_x)) + 1
        nu = kappa_x - sum(2**(i-1) for i in range(1, rho + 1))
        eta = math.floor(nu / mu_x)
        
        c_x = []
        for i in range(1, rho + 1):
            c_x.append(2**(i-1))
        for i in range(eta):
            c_x.append(mu_x)
        
        remainder = nu - eta * mu_x
        if remainder != 0:
            c_x.append(remainder)
        
        print(f"    Coefficients: {c_x}")
        return c_x

def create_bounded_encoding_system(kappa_vec, mu_vec):
    """Create complete encoding system"""
    print(f"\n=== CREATING ENCODING SYSTEM ===")
    
    all_coefficients = []
    widths = []
    
    for i in range(len(kappa_vec)):
        coeffs = bounded_coefficient_encoding(kappa_vec[i], mu_vec[i])
        all_coefficients.append(coeffs)
        widths.append(len(coeffs))
    
    total_binary_vars = sum(widths)
    print(f"Total binary variables: {total_binary_vars}")
    
    return all_coefficients, widths, total_binary_vars

def create_bounded_encoding_matrix(n_variables, all_coefficients, widths):
    """Create encoding matrix C where x = C @ y"""
    total_binary_vars = sum(widths)
    C = np.zeros((n_variables, total_binary_vars))
    
    start_idx = 0
    for i in range(n_variables):
        end_idx = start_idx + widths[i]
        C[i, start_idx:end_idx] = all_coefficients[i]
        start_idx = end_idx
    
    return C

# Create encoding
kappa_vec = D.copy()
mu_vec = [4, 4, 4]  # Smaller encoding for easier debugging

coefficients, widths, total_vars = create_bounded_encoding_system(kappa_vec, mu_vec)
C = create_bounded_encoding_matrix(n_products, coefficients, widths)

print(f"Encoding matrix C shape: {C.shape}")
print(f"Encoding matrix C:\n{C}")

# ===========================================
# QUBO FORMULATION (FIXED)
# ===========================================

def create_qubo_formulation_fixed(C, A, R, net_profit, penalty_multiplier=5):
    """
    Create FIXED QUBO formulation that properly handles both Q and h terms
    """
    print(f"\n=== CREATING FIXED QUBO FORMULATION ===")
    
    n_products, total_binary_vars = C.shape
    n_raw_materials = A.shape[0]
    
    # Step 1: Transform objective to binary space
    linear_objective = -(net_profit @ C)  # Negative for minimization
    print(f"Linear objective coefficients: {linear_objective}")
    
    # Step 2: Choose reasonable penalty weights
    max_profit = np.max(np.abs(net_profit))
    penalty_weights = penalty_multiplier * max_profit * np.ones(n_raw_materials)
    print(f"Penalty weights: {penalty_weights}")
    
    # Step 3: Transform constraints to binary space
    A_binary = A @ C
    print(f"Constraint matrix in binary space A_binary shape: {A_binary.shape}")
    
    # Step 4: Construct QUBO matrix and linear terms
    Q = np.zeros((total_binary_vars, total_binary_vars))
    h = linear_objective.copy()
    
    for i in range(n_raw_materials):
        P_i = penalty_weights[i]
        a_i = A_binary[i, :]
        
        # Quadratic penalty terms
        Q += P_i * np.outer(a_i, a_i)
        
        # Linear penalty terms
        h -= 2 * P_i * R[i] * a_i
    
    # Make Q upper triangular
    Q = np.triu(Q)
    
    print(f"QUBO matrix Q shape: {Q.shape}")
    print(f"Linear coefficients h shape: {h.shape}")
    print(f"Q matrix sparsity: {np.count_nonzero(Q)} / {Q.size} = {np.count_nonzero(Q)/Q.size:.3f}")
    
    return Q, h, penalty_weights

def convert_to_dimod_qubo(Q, h):
    """
    Convert Q matrix and h vector to proper dimod QUBO format
    """
    print(f"\n=== CONVERTING TO DIMOD QUBO FORMAT ===")
    
    qubo_dict = {}
    n_vars = len(h)
    
    # Add linear terms
    for i in range(n_vars):
        if h[i] != 0:
            qubo_dict[(i, i)] = h[i]
    
    # Add quadratic terms (upper triangular only)
    for i in range(n_vars):
        for j in range(i+1, n_vars):
            if Q[i, j] != 0:
                qubo_dict[(i, j)] = Q[i, j]
    
    print(f"QUBO dictionary has {len(qubo_dict)} terms")
    print(f"Linear terms: {sum(1 for (i,j) in qubo_dict.keys() if i == j)}")
    print(f"Quadratic terms: {sum(1 for (i,j) in qubo_dict.keys() if i != j)}")
    
    return qubo_dict

# Create the FIXED QUBO formulation
Q, h, penalty_weights = create_qubo_formulation_fixed(C, A, R, net_profit, penalty_multiplier=5)

# Convert to proper dimod format
qubo_dict = convert_to_dimod_qubo(Q, h)

# ===========================================
# FEASIBILITY CHECK
# ===========================================

def quick_feasibility_check(max_evaluations=1000):
    """Quick feasibility check with smaller encoding"""
    print(f"\n=== QUICK FEASIBILITY CHECK ===")
    print(f"Checking {max_evaluations:,} random solutions...")
    
    feasible_count = 0
    feasible_solutions = []
    n_bits = len(h)
    
    for i in range(max_evaluations):
        y = np.random.randint(0, 2, size=n_bits)
        x = C @ y
        
        demand_ok = np.all(x <= D)
        raw_ok = np.all(A @ x <= R)
        
        if demand_ok and raw_ok:
            feasible_count += 1
            profit = net_profit @ x
            feasible_solutions.append((y.copy(), x.copy(), profit))
            print(f"  Found feasible solution {feasible_count}: x={x}, profit={profit:.2f}")
    
    print(f"Feasible solutions found: {feasible_count}/{max_evaluations} ({100*feasible_count/max_evaluations:.2f}%)")
    return feasible_count > 0, feasible_solutions

# Run feasibility check
has_feasible, feasible_solutions = quick_feasibility_check(max_evaluations=5000)

# ===========================================
# SIMULATED ANNEALING (FIXED)
# ===========================================

def run_simulated_annealing_fixed(qubo_dict, num_runs=10):
    """
    Run simulated annealing with proper QUBO format
    """
    print(f"\n=== RUNNING SIMULATED ANNEALING (FIXED) ===")
    
    sampler = SimulatedAnnealingSampler()
    objectives = []
    solutions = []
    
    for i in range(num_runs):
        # Sample using the QUBO dictionary
        sampleset = sampler.sample_qubo(qubo_dict, num_reads=100)
        
        best_sample = sampleset.first
        objectives.append(best_sample.energy)
        
        # Convert sample to binary array
        n_vars = len(h)
        y_solution = np.array([best_sample.sample.get(i, 0) for i in range(n_vars)])
        solutions.append(y_solution)
        
        print(f"Run {i+1}: Energy = {best_sample.energy:.2f}")
    
    # Find best solution
    best_idx = np.argmin(objectives)
    best_y = solutions[best_idx]
    best_objective = objectives[best_idx]
    
    print(f"\nBest solution found:")
    print(f"Objective: {best_objective}")
    print(f"Binary solution: {best_y}")
    
    # Decode to production quantities
    best_x = C @ best_y
    best_profit = net_profit @ best_x
    
    print(f"Production quantities: {best_x}")
    print(f"Profit: {best_profit}")
    
    # Check feasibility
    demand_ok = np.all(best_x <= D)
    raw_usage = A @ best_x
    raw_ok = np.all(raw_usage <= R)
    
    print(f"Demand constraints satisfied: {demand_ok}")
    print(f"Raw material constraints satisfied: {raw_ok}")
    if not raw_ok:
        violations = raw_usage - R
        print(f"Raw material violations: {violations}")
    
    return best_y, best_objective, best_x, best_profit

# Run the fixed simulated annealing
if len(qubo_dict) > 0:
    best_y, best_obj, best_x, best_profit = run_simulated_annealing_fixed(qubo_dict, num_runs=10)
    
    # Compare with known feasible solutions
    if feasible_solutions:
        print(f"\n=== COMPARISON WITH KNOWN FEASIBLE SOLUTIONS ===")
        best_known_profit = max(sol[2] for sol in feasible_solutions)
        print(f"Best known feasible profit: {best_known_profit:.2f}")
        print(f"SA found profit: {best_profit:.2f}")
        
        if best_profit >= best_known_profit * 0.9:
            print("✓ Simulated annealing found near-optimal solution!")
        else:
            print("⚠ Simulated annealing may have found suboptimal solution")
else:
    print("ERROR: QUBO dictionary is empty!")

print(f"\n=== SUMMARY ===")
print(f"Problem size: {n_products} products, {n_raw_materials} raw materials")
print(f"Binary variables: {total_vars}")
print(f"Net profits: {net_profit}")
print(f"Feasible solutions exist: {has_feasible}")

=== CREATING REALISTIC PROBLEM INSTANCE ===
Selling prices S: [100. 120.  80.]
Raw material costs C: [1. 1. 2. 2. 3. 1. 2.]
Raw material limits R: [100. 120.  80.  90. 150. 200. 180.]
Demand limits D: [10 10 10]
Requirements matrix A (raw_materials x products):
[[2. 5. 4.]
 [3. 1. 1.]
 [1. 5. 4.]
 [4. 1. 5.]
 [5. 2. 1.]
 [1. 2. 3.]
 [3. 2. 4.]]
Net profit per unit: [63. 90. 43.]

=== CREATING ENCODING SYSTEM ===
  Encoding variable with κ=10, μ=4
    Using bounded-coefficient encoding
    Coefficients: [1, 2, 4, np.int64(3)]
  Encoding variable with κ=10, μ=4
    Using bounded-coefficient encoding
    Coefficients: [1, 2, 4, np.int64(3)]
  Encoding variable with κ=10, μ=4
    Using bounded-coefficient encoding
    Coefficients: [1, 2, 4, np.int64(3)]
Total binary variables: 12
Encoding matrix C shape: (3, 12)
Encoding matrix C:
[[1. 2. 4. 3. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 2. 4. 3. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 2. 4. 3.]]

=== CREATING FIXED QUBO FORMULATION ===
L

In [ ]:
import numpy as np
import math
import dimod
from neal import SimulatedAnnealingSampler

# ===========================================
# PROBLEM SETUP
# ===========================================

np.random.seed(42)
n_products = 3
n_raw_materials = 7

S = np.array([200., 240., 160.])  
A = np.array([[ 3.,  7.,  1.],
              [ 7.,  4., 10.],
              [ 5.,  7.,  7.],
              [ 7.,  6.,  9.],
              [ 4.,  3.,  5.],
              [ 8.,  3.,  2.],
              [ 4.,  1.,  9.]])
R = np.array([300., 360., 240., 270., 450., 600., 540.])  
C = np.array([3., 5., 3., 4., 2., 2., 3.])
D = np.array([30, 30, 30]) 

net_profit = S - A.T @ C
print(f"Net profit per unit: {net_profit}")

# ===========================================
# ENCODING SYSTEM
# ===========================================

def bounded_coefficient_encoding(kappa_x, mu_x):
    if kappa_x < 2**(math.floor(math.log2(mu_x)) + 1):
        log_kappa = math.floor(math.log2(kappa_x))
        binary_coeffs = [2**i for i in range(log_kappa + 1)]
        if sum(binary_coeffs[:-1]) < kappa_x:
            binary_coeffs[-1] = kappa_x - sum(binary_coeffs[:-1])
        return binary_coeffs
    else:
        rho = math.floor(math.log2(mu_x)) + 1
        nu = kappa_x - sum(2**(i-1) for i in range(1, rho + 1))
        eta = math.floor(nu / mu_x)
        
        c_x = []
        for i in range(1, rho + 1):
            c_x.append(2**(i-1))
        for i in range(eta):
            c_x.append(mu_x)
        
        remainder = nu - eta * mu_x
        if remainder != 0:
            c_x.append(remainder)
        
        return c_x

def create_bounded_encoding_system(kappa_vec, mu_vec):
    all_coefficients = []
    widths = []
    
    for i in range(len(kappa_vec)):
        coeffs = bounded_coefficient_encoding(kappa_vec[i], mu_vec[i])
        all_coefficients.append(coeffs)
        widths.append(len(coeffs))
    
    total_binary_vars = sum(widths)
    return all_coefficients, widths, total_binary_vars

def create_bounded_encoding_matrix(n_variables, all_coefficients, widths):
    total_binary_vars = sum(widths)
    C = np.zeros((n_variables, total_binary_vars))
    
    start_idx = 0
    for i in range(n_variables):
        end_idx = start_idx + widths[i]
        C[i, start_idx:end_idx] = all_coefficients[i]
        start_idx = end_idx
    
    return C

kappa_vec = D.copy()
mu_vec = [4, 4, 4] 

coefficients, widths, total_vars = create_bounded_encoding_system(kappa_vec, mu_vec)
C = create_bounded_encoding_matrix(n_products, coefficients, widths)

# ===========================================
# SCALED QUBO FORMULATION
# ===========================================

def create_scaled_qubo(C, A, R, net_profit, penalty_ratio=100):
    """
    Create QUBO with proper scaling to avoid numerical precision issues
    """
    n_products, total_vars = C.shape
    n_raw_materials = A.shape[0]

    # Scale profits to reasonable range (0-1000)
    profit_scale = 1000 / np.max(np.abs(net_profit))
    scaled_net_profit = net_profit * profit_scale
    
    # Linear objective (profit)
    linear_objective = -(scaled_net_profit @ C)
    
    # Penalty weights - use ratio instead of absolute multiplier
    max_scaled_profit = np.max(np.abs(scaled_net_profit))
    penalty_weights = penalty_ratio * max_scaled_profit * np.ones(n_raw_materials)
    
    # Transform constraints to binary space
    A_binary = A @ C
    
    # Build QUBO
    Q = np.zeros((total_vars, total_vars))
    h = linear_objective.copy()
    
    for i in range(n_raw_materials):
        P_i = penalty_weights[i]
        a_i = A_binary[i, :]
        
        # Quadratic penalty terms
        Q += P_i * np.outer(a_i, a_i)
        
        # Linear penalty terms
        h -= 2 * P_i * R[i] * a_i
    
    Q = np.triu(Q)
    
    print(f"Scaling factor: {profit_scale:.2f}")
    print(f"Scaled profits: {scaled_net_profit}")
    print(f"Penalty weights: {penalty_weights[0]:.2f}")
    print(f"Linear term range: [{np.min(h):.2f}, {np.max(h):.2f}]")
    print(f"Quadratic term range: [{np.min(Q[Q>0]):.2f}, {np.max(Q):.2f}]")
    
    return Q, h, penalty_weights, profit_scale

Q, h, penalty_weights, profit_scale = create_scaled_qubo(C, A, R, net_profit, penalty_ratio=50)

# ===========================================
# QUBO DICTIONARY CONSTRUCTION
# ===========================================

def create_qubo_dict(Q, h):
    qubo_dict = {}
    
    # Combine diagonal terms properly
    for i in range(len(h)):
        total_diagonal = h[i] + Q[i, i]
        if abs(total_diagonal) > 1e-10:  # Avoid numerical noise
            qubo_dict[(i, i)] = total_diagonal
    
    # Add off-diagonal terms
    for i in range(len(h)):
        for j in range(i + 1, len(h)):
            if abs(Q[i, j]) > 1e-10:
                qubo_dict[(i, j)] = Q[i, j]
    
    return qubo_dict

qubo_dict = create_qubo_dict(Q, h)
print(f"QUBO dictionary size: {len(qubo_dict)}")

# ===========================================
# SIMULATED ANNEALING
# ===========================================

def run_enhanced_annealing(qubo_dict, num_runs=20):
    """Run SA with multiple parameter settings"""
    
    sampler = SimulatedAnnealingSampler()
    all_results = []
    
    # Try different parameter combinations
    parameter_sets = [
        {'num_reads': 1000, 'num_sweeps': 10000},
        {'num_reads': 500, 'num_sweeps': 20000}, 
        {'num_reads': 100, 'num_sweeps': 50000},
        {'num_reads': 1000, 'num_sweeps': 5000, 'beta_range': [0.1, 10.0]},
        {'num_reads': 1000, 'num_sweeps': 5000, 'beta_schedule_type': 'linear'}
    ]
    
    best_overall_energy = float('inf')
    best_overall_sample = None
    
    for i, params in enumerate(parameter_sets):
        print(f"Parameter set {i+1}: {params}")
        
        try:
            sampleset = sampler.sample_qubo(qubo_dict, **params)
            
            best_energy = sampleset.first.energy
            best_sample = sampleset.first.sample
            
            # Convert to binary array
            y_solution = np.array([best_sample.get(j, 0) for j in range(len(h))])
            x_solution = C @ y_solution
            
            print(f"  Energy: {best_energy:.2f}, Production: {x_solution}")
            
            all_results.append({
                'energy': best_energy,
                'y': y_solution,
                'x': x_solution,
                'params': params
            })
            
            if best_energy < best_overall_energy:
                best_overall_energy = best_energy
                best_overall_sample = y_solution
                
        except Exception as e:
            print(f"  Failed: {e}")
    
    return best_overall_sample, best_overall_energy, all_results

# ===========================================
# FEASIBILITY-FIRST SEARCH
# ===========================================

def find_feasible_solutions(num_trials=50000):
    """Find feasible solutions first, then optimize among them"""
    print(f"\nSearching for feasible solutions in {num_trials} random trials...")
    
    feasible_solutions = []
    n_vars = len(h)
    
    for i in range(num_trials):
        # Random binary solution
        y = np.random.randint(0, 2, size=n_vars)
        x = C @ y
        
        # Check feasibility
        demand_ok = np.all(x <= D)
        raw_ok = np.all(A @ x <= R)
        
        if demand_ok and raw_ok:
            profit = net_profit @ x
            energy = y.T @ Q @ y + h.T @ y
            
            feasible_solutions.append({
                'y': y.copy(),
                'x': x.copy(), 
                'profit': profit,
                'energy': energy
            })
            
            if len(feasible_solutions) <= 10:  # Print first 10
                print(f"  Feasible #{len(feasible_solutions)}: x={x}, profit={profit:.2f}")
    
    print(f"Found {len(feasible_solutions)} feasible solutions")
    
    if feasible_solutions:
        # Find best feasible by profit
        best_feasible = max(feasible_solutions, key=lambda s: s['profit'])
        print(f"Best feasible solution: x={best_feasible['x']}, profit={best_feasible['profit']:.2f}")
        return best_feasible
    else:
        print("No feasible solutions found!")
        return None

# ===========================================
# RUN OPTIMIZATION
# ===========================================

# First, try to find feasible solutions
best_feasible = find_feasible_solutions(num_trials=100000)

# Run enhanced simulated annealing
"""print(f"\n=== RUNNING ENHANCED SIMULATED ANNEALING ===")
best_y, best_energy, all_results = run_enhanced_annealing(qubo_dict)

best_x = C @ best_y
best_profit = net_profit @ best_x

print(f"\n=== BEST SA RESULT ===")
print(f"Energy: {best_energy:.2f}")
print(f"Production: {best_x}")
print(f"Profit: {best_profit:.2f}")

# Check constraints
demand_ok = np.all(best_x <= D)
raw_usage = A @ best_x
raw_ok = np.all(raw_usage <= R)

print(f"Demand satisfied: {demand_ok}")
print(f"Raw material satisfied: {raw_ok}")

if not raw_ok:
    violations = np.maximum(0, raw_usage - R)
    print(f"Violations: {violations}")
"""
# ===========================================
# VERIFICATION FUNCTION
# ===========================================

def verify_solution(y, x, energy, Q, h, description=""):
    print(f"\n=== VERIFICATION: {description} ===")
    
    # Check decoding
    decoded_x = C @ y
    print(f"Decoding correct: {np.allclose(decoded_x, x)}")
    
    # Check energy calculation
    manual_energy = y.T @ Q @ y + h.T @ y
    print(f"Energy correct: {np.isclose(manual_energy, energy)}")
    print(f"  Reported: {energy:.2f}")
    print(f"  Calculated: {manual_energy:.2f}")
    
    # Check constraints
    demand_ok = np.all(x <= D)
    raw_usage = A @ x
    raw_ok = np.all(raw_usage <= R)
    
    print(f"Production: {x}")
    print(f"Profit: {net_profit @ x:.2f}")
    print(f"Demand satisfied: {demand_ok}")
    print(f"Raw material satisfied: {raw_ok}")
    
    if not raw_ok:
        violations = np.maximum(0, raw_usage - R)
        print(f"Constraint violations: {violations}")
    
    return demand_ok and raw_ok

# Verify results
"""
verify_solution(best_y, best_x, best_energy, Q, h, "SA Solution")
"""

if best_feasible:
    verify_solution(best_feasible['y'], best_feasible['x'], 
                   best_feasible['energy'], Q, h, "Best Feasible")

# ===========================================
# SUMMARY AND RECOMMENDATIONS
# ===========================================

"""print(f"\n=== SUMMARY ===")
print(f"Problem scaling applied: profit scale = {profit_scale:.2f}")
print(f"Penalty ratio used: 50")

if best_feasible and raw_ok:
    print("✓ Found feasible solution via SA")
elif best_feasible:
    print("⚠ SA found infeasible solution, but feasible solutions exist")
    print("  Recommendation: Increase penalty ratio or try different SA parameters")
else:
    print("✗ No feasible solutions found")
    print("  Recommendation: Problem may be over-constrained")

print(f"\nCoefficient ranges after scaling:")
print(f"  Linear terms: [{np.min(h):.2f}, {np.max(h):.2f}]")
print(f"  Quadratic terms: [{np.min(Q[Q>0]):.2f}, {np.max(Q):.2f}]")
ratio = np.max(Q) / max(abs(np.min(h)), abs(np.max(h)))
print(f"  Quad/Linear ratio: {ratio:.2f} (should be 10-1000 for good performance)")"""

Net profit per unit: [ 77. 139.   9.]
Scaling factor: 7.19
Scaled profits: [ 553.95683453 1000.           64.74820144]
Penalty weights: 50000.00
Linear term range: [-6528000258.99, -1053001000.00]
Quadratic term range: [8300000.00, 272800000.00]
QUBO dictionary size: 378

Searching for feasible solutions in 100000 random trials...
  Feasible #1: x=[11.  5. 12.], profit=1650.00
  Feasible #2: x=[18. 12.  8.], profit=3126.00
  Feasible #3: x=[12. 11.  3.], profit=2480.00
  Feasible #4: x=[ 8.  9. 13.], profit=1984.00
  Feasible #5: x=[13. 11. 12.], profit=2638.00
  Feasible #6: x=[ 6. 11. 15.], profit=2126.00
  Feasible #7: x=[ 3. 11. 17.], profit=1913.00
  Feasible #8: x=[9. 7. 5.], profit=1711.00
  Feasible #9: x=[12.  4. 11.], profit=1579.00
  Feasible #10: x=[ 3. 16. 16.], profit=2599.00
Found 17982 feasible solutions
Best feasible solution: x=[17. 22.  0.], profit=4367.00

=== VERIFICATION: Best Feasible ===
Decoding correct: True
Energy correct: True
  Reported: -41628581417.27
  C

'print(f"\n=== SUMMARY ===")\nprint(f"Problem scaling applied: profit scale = {profit_scale:.2f}")\nprint(f"Penalty ratio used: 50")\n\nif best_feasible and raw_ok:\n    print("✓ Found feasible solution via SA")\nelif best_feasible:\n    print("⚠ SA found infeasible solution, but feasible solutions exist")\n    print("  Recommendation: Increase penalty ratio or try different SA parameters")\nelse:\n    print("✗ No feasible solutions found")\n    print("  Recommendation: Problem may be over-constrained")\n\nprint(f"\nCoefficient ranges after scaling:")\nprint(f"  Linear terms: [{np.min(h):.2f}, {np.max(h):.2f}]")\nprint(f"  Quadratic terms: [{np.min(Q[Q>0]):.2f}, {np.max(Q):.2f}]")\nratio = np.max(Q) / max(abs(np.min(h)), abs(np.max(h)))\nprint(f"  Quad/Linear ratio: {ratio:.2f} (should be 10-1000 for good performance)")'